In [ ]:
!pip install ultralytics

In [ ]:
!pip install cvzone

In [ ]:
!pip install pytesseract

In [1]:
import cv2
import pandas as pd
from ultralytics import YOLO
import cvzone
import numpy as np
import pytesseract
from datetime import datetime

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\pc\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

model = YOLO('best.pt')

In [6]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        point = [x, y]
        print(point)

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap = cv2.VideoCapture('mycarplate.mp4')

my_file = open("plateClass.txt", "r")
data = my_file.read()
class_list = data.split("\n") 

area = [(27, 417), (16, 456), (1015, 451), (992, 417)]

count = 0
list1 = []
processed_numbers = set()

# Open file for writing car plate data
with open("car_plate_data.txt", "a") as file:
    file.write("NumberPlate\tDate\tTime\n")  # Writing column headers

while True:    
    ret, frame = cap.read()
    count += 1
    if count % 3 != 0:
        continue
    if not ret:
       break
   
    frame = cv2.resize(frame, (1020, 500))
    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.cpu()
    px = pd.DataFrame(a).astype("float")
   
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        
        d = int(row[5])
        c = class_list[d]
        cx = int(x1 + x2) // 2
        cy = int(y1 + y2) // 2
        result = cv2.pointPolygonTest(np.array(area, np.int32), ((cx, cy)), False)
        if result >= 0:
           crop = frame[y1:y2, x1:x2]
           gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
           gray = cv2.bilateralFilter(gray, 10, 20, 20)

           text = pytesseract.image_to_string(gray).strip()
           text = text.replace('(', '').replace(')', '').replace(',', '').replace('|', '').replace('"', '')
           if text not in processed_numbers:
              processed_numbers.add(text) 
              list1.append(text)
              current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
              with open("car_plate_data.txt", "a") as file:
                   file.write(f"{text}\t{current_datetime}\n")
                   cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
                   cv2.imshow('crop', crop)

      
    cv2.polylines(frame, [np.array(area, np.int32)], True, (255, 0, 0), 2)
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()    
cv2.destroyAllWindows()


0: 416x800 (no detections), 143.5ms
Speed: 7.0ms preprocess, 143.5ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 1 car plate, 135.0ms
Speed: 2.0ms preprocess, 135.0ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 1 car plate, 137.5ms
Speed: 2.0ms preprocess, 137.5ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 1 car plate, 136.5ms
Speed: 2.5ms preprocess, 136.5ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 1 car plate, 140.1ms
Speed: 2.0ms preprocess, 140.1ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 1 car plate, 133.5ms
Speed: 3.0ms preprocess, 133.5ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 800)
[477, 130]

0: 416x800 1 car plate, 133.5ms
Speed: 2.0ms preprocess, 133.5ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 1 car plate, 133.5ms
Speed: 3.0ms preprocess, 133.5ms in